In [1]:
import itertools
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
#read network information
network_file = open('links.txt','r',encoding='UTF-8')
line = network_file.readline()
cost = {}
nodes = []
while(len(line)):
    line = line.strip('\n')
    data = line.split()
    from_node = data[0]
    to_node = data[1]
    fuel_cost = float(data[3])
    time_cost = float(data[4])
    toll_cost = float(data[5])
    cost[from_node,to_node] = [fuel_cost,time_cost,toll_cost]
    nodes.append(from_node)
    nodes.append(to_node)
    line = network_file.readline()
network_file.close()
links,fuel_costs,time_costs,toll_costs = multidict(cost)
nodes = list(set(nodes))

In [3]:
# read vehicles information(origin,destination,start time, deadline)
vehicles_file = open('vehicles.txt','r',encoding='UTF-8')
line = vehicles_file.readline()
vehicles = {}
pre_index_of_v_w = {}

while(len(line)):
    line = line.strip('\n')
    data = line.split()
    origin = data[4]
    destination = data[5]
    departure_time = float(data[2])
    #deadline = float(data[5])
    vehicle = data[1]
    vehicles[vehicle] = [origin,destination,departure_time]
    #prepare for index of (v,w)
    pre_index_of_v_w[int(data[0])]=vehicle
    
    line = vehicles_file.readline()
vehicles_file.close()
    
    


In [4]:
vehicles

{'V447837': ['富士', '浜松西', 55.0],
 'V285613_2': ['沼津', '京都南', 53.0],
 'V397266_3': ['竜王', '西宮', 36.0],
 'V293886_2': ['袋井', '吹田', 46.0],
 'V288981_3': ['清水', '茨木', 1.0],
 'V334983_3': ['豊川', '西宮', 56.0],
 'V296199_3': ['袋井', '京都南', 25.0],
 'V289130_3': ['静岡', '岡崎', 30.0],
 'V447837_2': ['富士', '浜松西', 20.0],
 'V296288_2': ['袋井', '茨木', 28.0],
 'V287029_2': ['富士', '岡崎', 39.0],
 'V331444_2': ['小牧', '京都南', 9.0],
 'V294461_3': ['掛川', '小牧', 43.0],
 'V260963_2': ['大垣', '西宮', 26.0],
 'V329472_3': ['一宮', '西宮', 18.0],
 'V199214_3': ['大井松田', '京都南', 13.0],
 'V285615': ['沼津', '京都南', 29.0],
 'V306658': ['沼津', '関ヶ原', 47.0],
 'V288816_2': ['袋井', '岐阜羽島', 7.0],
 'V294516_2': ['掛川', '茨木', 1.0],
 'V307800_2': ['袋井', '茨木', 52.0],
 'V294545_3': ['掛川', '西宮', 54.0],
 'V285271_2': ['袋井', '茨木', 24.0],
 'V285271_3': ['袋井', '茨木', 52.0],
 'V295731_3': ['袋井', '吹田', 20.0]}

In [5]:
#construct follow index(v,w)
index_v_w = []
for i in pre_index_of_v_w:
    for j in pre_index_of_v_w:
        if i>j:
            vw = (pre_index_of_v_w[i],pre_index_of_v_w[j])
            index_v_w.append(vw)

In [6]:
# Set other parameters
M = 500
Coef = 0.1
Alpha = 50

In [7]:
#Add Vars
m = Model('truck_platooning')
f = m.addVars(vehicles.keys(),links,vtype = GRB.BINARY,name = 'f')
q = m.addVars(index_v_w,links,vtype = GRB.BINARY, name = 'q')
e = m.addVars(vehicles.keys(),links,vtype = GRB.CONTINUOUS,name = 'e')
var_w = m.addVars(vehicles.keys(),nodes,vtype = GRB.CONTINUOUS,name = 'w') # special name for differentiating from "Vehicle w"

Academic license - for non-commercial use only - expires 2021-12-31
Using license file C:\Users\FENG\gurobi.lic


In [8]:
#ADD CONSTRAINTS
# constraint 1: node outflow must equal inflows
for v in vehicles.keys():
    for i,j in links:
        if i == vehicles[v][0]:
            m.addConstr(f.sum(v,i,'*')-f.sum(v,'*',i) == 1)
        elif i == vehicles[v][1]:
            m.addConstr(f.sum(v,i,'*')-f.sum(v,'*',i) == -1)
        else:
            m.addConstr(f.sum(v,i,'*')-f.sum(v,'*',i) == 0)

In [9]:
for v,w,i,j in q.keys():
# constraint 2: when platooning, enter times are equal.
    m.addConstr(e[v,i,j]-e[w,i,j] >= -M*(1-q[v,w,i,j]))
    m.addConstr(e[v,i,j]-e[w,i,j] <= M*(1-q[v,w,i,j]))
# constraint 3-1: only one vehicle can follow    
    m.addConstr(q.sum('*',w,i,j) <= 1)
#constraint 3-2 : only one vehicle can lead
    m.addConstr(q.sum(v,'*',i,j) <= 1)
# constraint 4: platooning requires flow for the leader and followers
    m.addConstr(q[v,w,i,j] <= f[w,i,j])
    m.addConstr(q[v,w,i,j] <= f[v,i,j])

In [10]:
# constraint 5: set origin enter time
for v,j in var_w.keys():
    if (vehicles[v][0],j) in links:
        m.addConstr(e[v,vehicles[v][0],j] - vehicles[v][2] - var_w[v,vehicles[v][0]] >= -M*(1-f[v,vehicles[v][0],j]))
        m.addConstr(e[v,vehicles[v][0],j] - vehicles[v][2] - var_w[v,vehicles[v][0]] <=  M*(1-f[v,vehicles[v][0],j]))

In [11]:
#constraint 6: set arrival time ("waiting" at destination means arrive before deadline)
#for v,i in var_w.keys():
    #if (i,vehicles[v][1]) in links:
        #m.addConstr(vehicles[v][3] - e[v,i,vehicles[v][1]]-var_w[v,vehicles[v][1]]-time_costs[i,vehicles[v][1]]*f[v,i,vehicles[v][1]] >= -M*(1-f[v,i,vehicles[v][1]]))
        #m.addConstr(vehicles[v][3] - e[v,i,vehicles[v][1]]-var_w[v,vehicles[v][1]]-time_costs[i,vehicles[v][1]]*f[v,i,vehicles[v][1]] <=  M*(1-f[v,i,vehicles[v][1]]))


In [12]:
# constraint 6: waiting time at destination is 0
for v in vehicles.keys():
    m.addConstr(var_w[v,vehicles[v][1]] == 0 )

In [13]:
# constraint 7: set intermediate enter time
for v,i,j in f.keys():
    if i != vehicles[v][0] and i!= vehicles[v][1]:
        link_k_i = links.select('*',i)
        for k,i in link_k_i:
            if k != j:
                m.addConstr(e[v,i,j]-e[v,k,i]-var_w[v,i]-time_costs[k,i]*f[v,k,i] >= -M*(2-f[v,i,j]-f[v,k,i]))
                m.addConstr(e[v,i,j]-e[v,k,i]-var_w[v,i]-time_costs[k,i]*f[v,k,i] <=  M*(2-f[v,i,j]-f[v,k,i]))
# constraint 8: if no flow, enter time must be zero.
    m.addConstr(e[v,i,j] <= M*f[v,i,j])


In [14]:
# constraint 9: if no flow, wait time must be zero.
for v,i in var_w.keys():
    m.addConstr(var_w[v,i] <= M*(f.sum(v,i,'*')+f.sum(v,'*',i)))

In [15]:
#Objective function
#obj = quicksum(fuel_costs[i,j]*(f[v,i,j]-Coef*q.sum(v,'*',i,j))   for v,i,j in f.keys()) + quicksum(toll_costs[i,j]*f[v,i,j] for v,i,j in f.keys()) + Alpha * quicksum(time_costs[i,j]*f[v,i,j] for v,i,j in f.keys()) + Alpha * quicksum(var_w[v,i]*f[v,i,j] for v,i,j in f.keys() )
#m.setObjective(obj,GRB.MINIMIZE)

In [16]:
#simpfied Objective function
obj = quicksum(4.132845571*fuel_costs[i,j]*f[v,i,j] - Coef*fuel_costs[i,j]*q.sum(v,'*',i,j) + Alpha*f[v,i,j]*var_w[v,i]   for v,i,j in f.keys())  
m.setObjective(obj,GRB.MINIMIZE)

In [17]:
#set model parameters
m.params.TimeLimit = 200

Changed value of parameter TimeLimit to 200.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [18]:
links

<gurobi.tuplelist (37 tuples, 2 values each):
 ( 東京   , 横浜町田 )
 ( 横浜町田 , 厚木   )
 ( 厚木   , 秦野中井 )
 ( 秦野中井 , 大井松田 )
 ( 大井松田 , 沼津   )
 ( 沼津   , 富士   )
 ( 富士   , 清水   )
 ( 清水   , 静岡   )
 ( 静岡   , 焼津   )
 ( 焼津   , 吉田   )
 ( 吉田   , 掛川   )
 ( 掛川   , 袋井   )
 ( 袋井   , 浜松   )
 ( 浜松   , 浜松西  )
 ( 浜松西  , 豊川   )
 ( 豊川   , 音羽蒲郡 )
 ( 音羽蒲郡 , 岡崎   )
 ( 岡崎   , 豊田   )
 ( 豊田   , 名古屋  )
 ( 名古屋  , 春日井  )
 ( 春日井  , 小牧   )
 ( 小牧   , 一宮   )
 ( 一宮   , 岐阜羽島 )
 ( 岐阜羽島 , 大垣   )
 ( 大垣   , 関ヶ原  )
 ( 関ヶ原  , 彦根   )
 ( 彦根   , 八日市  )
 ( 八日市  , 竜王   )
 ( 竜王   , 栗東   )
 ( 栗東   , 瀬田東  )
 ( 瀬田東  , 京都南  )
 ( 京都南  , 大山崎  )
 ( 大山崎  , 茨木   )
 ( 茨木   , 吹田   )
 ( 吹田   , 豊中   )
 ( 豊中   , 尼崎   )
 ( 尼崎   , 西宮   )
>

In [19]:
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 71185 rows, 13900 columns and 488800 nonzeros
Model fingerprint: 0xe0e25585
Model has 925 quadratic objective terms
Variable types: 1875 continuous, 12025 integer (12025 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+01, 5e+03]
  QObjective range [1e+02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 63403 rows and 10155 columns
Presolve time: 0.62s
Presolved: 7782 rows, 3745 columns, 54358 nonzeros
Variable types: 445 continuous, 3300 integer (3300 binary)
Found heuristic solution: objective 735834.34899

Root relaxation: objective 7.207629e+05, 4057 iterations, 0.16 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 720762.903  

In [20]:
#total cost
m.Objval

729862.3764057833

In [21]:
#total time cost
total_time = 0
for v,i,j in f.keys():
    total_time = time_costs[i,j] * f[v,i,j].x + total_time + var_w[v,i].x * f[v,i,j].x
print(total_time)

4991.950000000145


In [22]:
#total distance
total_distance = 0
for v,i,j in f.keys():
    total_distance = time_costs[i,j] / 60 * 80 * f[v,i,j].x + total_distance
print(total_distance)

6578.099999999994


In [23]:
#platoon distance
platoon_distance = 0
for v,w,i,j in q.keys():
    platoon_distance = time_costs[i,j] / 60 * 80 * q[v,w,i,j].x + platoon_distance
print(platoon_distance)

3585.8999999999974


In [24]:
#input original data
O_time_cost = 4933.572484




O_distance = 6578.096645

 



O_toll_cost = 311736





O_fuel_cost = 178245.1994

 



O_total_cost = O_time_cost * 50 + O_toll_cost + O_fuel_cost

In [25]:
# cost saving rate
cr = (O_total_cost - m.Objval) / O_total_cost
print(cr)

0.009227389598903403


In [26]:
# matching rate
mr = platoon_distance/total_distance
print(mr)

0.5451270123591919


In [27]:
# delay rate
dlr = (total_time - O_time_cost) / O_time_cost
print(dlr)

0.011832706662255067


In [28]:
# detour rate
dtr = (total_distance - O_distance ) / O_distance
print(dtr)

5.100259505797532e-07


In [29]:
# fuel saving rate
fuel_cost = m.Objval - total_distance * 47.39 - total_time * 50
fsr = (O_fuel_cost - fuel_cost) / O_fuel_cost
print(fsr)

0.05451188602515392


In [30]:
fuel_cost

168528.71740577638

In [31]:
fuel_cost_2 =  total_distance*126/4.65 - 0.1 * platoon_distance*126/4.65
fuel_cost

168528.71740577638

In [32]:
idle = [v.X for v in var_w.values()]

In [33]:
idle

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.842170943040401e-14,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.2538415578310378e-11,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.1174173525650986e-11,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 6.75,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0378365661599673e-11,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.7853275241795927e-12,

In [34]:
'''
q_result = {}
for i,j,k,l in q.keys():
    q_result[(i,j,k,l)] = q[i,j,k,l].x
q_result
'''

'\nq_result = {}\nfor i,j,k,l in q.keys():\n    q_result[(i,j,k,l)] = q[i,j,k,l].x\nq_result\n'

In [35]:

w_result = {}
for i,j in var_w.keys():
    w_result[(i,j)] = var_w[i,j].x
w_result


{('V447837', '浜松'): 0.0,
 ('V447837', '栗東'): 0.0,
 ('V447837', '豊川'): 0.0,
 ('V447837', '豊中'): 0.0,
 ('V447837', '岐阜羽島'): 0.0,
 ('V447837', '焼津'): 0.0,
 ('V447837', '茨木'): 0.0,
 ('V447837', '関ヶ原'): 0.0,
 ('V447837', '東京'): 0.0,
 ('V447837', '八日市'): 0.0,
 ('V447837', '一宮'): 0.0,
 ('V447837', '沼津'): 0.0,
 ('V447837', '秦野中井'): 0.0,
 ('V447837', '豊田'): 0.0,
 ('V447837', '尼崎'): 0.0,
 ('V447837', '掛川'): 0.0,
 ('V447837', '西宮'): 0.0,
 ('V447837', '大垣'): 0.0,
 ('V447837', '袋井'): 0.0,
 ('V447837', '大井松田'): 0.0,
 ('V447837', '小牧'): 0.0,
 ('V447837', '吹田'): 0.0,
 ('V447837', '大山崎'): 0.0,
 ('V447837', '岡崎'): 0.0,
 ('V447837', '静岡'): 0.0,
 ('V447837', '京都南'): 0.0,
 ('V447837', '清水'): 0.0,
 ('V447837', '横浜町田'): 0.0,
 ('V447837', '竜王'): 0.0,
 ('V447837', '富士'): 0.0,
 ('V447837', '音羽蒲郡'): 0.0,
 ('V447837', '名古屋'): 0.0,
 ('V447837', '厚木'): 0.0,
 ('V447837', '瀬田東'): 0.0,
 ('V447837', '浜松西'): 0.0,
 ('V447837', '吉田'): 0.0,
 ('V447837', '彦根'): 0.0,
 ('V447837', '春日井'): 0.0,
 ('V285613_2', '浜松'): 2.84217094

In [36]:
'''
for key,value in w_result.items():
    if value > 0:
        print(key,value)
'''

'\nfor key,value in w_result.items():\n    if value > 0:\n        print(key,value)\n'

In [37]:
formation_location = [ ]
for key,value in w_result.items():
    if value > 0:
        a_formation_location = [ ]
        key_list = list(key)
        key_list.append(value)
        formation_location.append(key_list)
formation_location

[['V285613_2', '浜松', 2.842170943040401e-14],
 ['V285613_2', '音羽蒲郡', 2.2538415578310378e-11],
 ['V293886_2', '一宮', 2.1174173525650986e-11],
 ['V293886_2', '袋井', 6.75],
 ['V293886_2', '音羽蒲郡', 2.0378365661599673e-11],
 ['V293886_2', '彦根', 2.7853275241795927e-12],
 ['V296199_3', '岐阜羽島', 2.6432189770275727e-12],
 ['V296199_3', '袋井', 2.999999999999588],
 ['V296199_3', '名古屋', 2.5011104298755527e-12],
 ['V296288_2', '豊川', 5.684341886080802e-14],
 ['V287029_2', '焼津', 1.8758328224066645e-12],
 ['V287029_2', '富士', 3.875],
 ['V331444_2', '一宮', 2.7749999999999773],
 ['V260963_2', '大垣', 6.84999999999998],
 ['V199214_3', '浜松', 2.842170943040401e-14],
 ['V199214_3', '大井松田', 3.7749999999999773],
 ['V199214_3', '音羽蒲郡', 2.2538415578310378e-11],
 ['V306658', '浜松', 2.842170943040401e-14],
 ['V306658', '沼津', 6.0],
 ['V306658', '音羽蒲郡', 2.2538415578310378e-11],
 ['V288816_2', '袋井', 3.75],
 ['V288816_2', '浜松西', 3.249999999999261],
 ['V294516_2', '関ヶ原', 1.3926637620897964e-12],
 ['V294516_2', '一宮', 4.5474735088

In [38]:
df1 = pd.DataFrame(formation_location)
df1.to_csv('fomation.csv')